# Philippine Family Income and Expenditure (FIES) Analysis
## Income Classification, Household Clustering, Anomaly Detection & Dimensionality Reduction

**Course:** AI 221 Classical Machine Learning  
**Data Source:** Kaggle - Family Income and Expenditure Survey (Philippine Statistics Authority)  
**Dataset Size:** 40,000+ households × 60 variables

### Project Overview
This project addresses the critical problem of **socio-economic classification in the Philippines** using machine learning methods covered in AI 221. We combine supervised and unsupervised learning to:

1. **Supervised Learning:** Predict household income levels from expenditure patterns
2. **Unsupervised Learning:** Discover natural household segments and detect anomalies
3. **Visualization:** Reduce dimensionality to understand the household landscape

---

## Project Objectives

- **Supervised:** Train classification models to predict income bracket based on household expenditures
- **Unsupervised:** Identify distinct household clusters and detect unusual spending patterns
- **Analysis:** Extract actionable insights about Filipino household economics

---

## Section 1: Data Loading and Exploratory Analysis

### Methodology

**Step 1.1: Data Loading**
- Load the FIES dataset from CSV file
- Check dataset shape, data types, and basic information
- Identify target variables and feature variables

**Step 1.2: Missing Values Analysis**
- Quantify missing values per column
- Decide on imputation strategy (mean, median, forward-fill, or removal)
- Document handling decisions

**Step 1.3: Exploratory Data Analysis (EDA)**
- Summary statistics (mean, std, min, max, quartiles)
- Distribution analysis of income and major expenditure categories
- Identify skewed distributions and outliers
- Correlation analysis between features
- Geographic and demographic breakdowns

**Step 1.4: Feature Understanding**
- Categorize variables: Demographics, Income sources, Expenditure categories
- Create income brackets/levels (e.g., Quintiles: Q1-Q5, or Low/Middle/High)
- Document all 60 variables and their roles

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
print("Libraries imported successfully!")

In [ ]:
# STEP 1: LOAD DATA
# Replace 'filepath' with actual Kaggle dataset path
# Download from: https://www.kaggle.com/datasets/grosvenpaul/family-income-and-expenditure

# df = pd.read_csv('path_to_dataset/Family Income and Expenditure.csv')
# For demonstration, we'll create sample structure

# Display basic information
print("=" * 80)
print("DATASET LOADING")
print("=" * 80)
print("\n# TODO: Load your dataset with:")
print("df = pd.read_csv('path_to/Family Income and Expenditure.csv')")
print("\n# After loading, run these exploratory commands:"
print("print(df.shape)")
print("print(df.info())")
print("print(df.head())")

### Data Exploration Code
After loading the dataset, perform these analyses:

In [ ]:
# STEP 2: MISSING VALUES ANALYSIS
# Check missing values
# missing = df.isnull().sum()
# missing_percent = (missing / len(df)) * 100
# missing_df = pd.DataFrame({'Missing_Count': missing, 'Percentage': missing_percent})
# print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Percentage', ascending=False))

print("Check for missing values in your dataset")
print("Handle missing values: drop or impute based on percentage missing")

In [ ]:
# STEP 3: SUMMARY STATISTICS
# print("\n=== SUMMARY STATISTICS ===")
# print(df.describe())
# print("\nData Types:")
# print(df.dtypes)

print("Summary Statistics Template:")
print("- Review mean, std, min, max for all features")
print("- Check for unrealistic values (negative income, etc.)")
print("- Identify potential data entry errors")

---

## Section 2: Income Level Classification (SUPERVISED LEARNING)

### Methodology Overview

**Goal:** Predict household income bracket from expenditure patterns using supervised classification.

**Approach:**
1. Create income target variable (income brackets/quintiles)
2. Select relevant features (expenditure categories, demographics)
3. Preprocess: Scale features, handle missing values
4. Split data: 80% train, 20% test
5. Train multiple classifiers
6. Evaluate and compare model performance

### Detailed Steps

#### Step 2.1: Create Target Variable - Income Brackets

**Method:** Create income quintiles (Q1-Q5) or categories (Low/Middle/High)

```python
# Option 1: Quintiles (5 classes)
df['income_quintile'] = pd.qcut(df['Total Income'], q=5, 
                                labels=['Q1_Lowest', 'Q2_Low', 'Q3_Middle', 'Q4_High', 'Q5_Highest'])

# Option 2: Simple categories (3 classes) - Better for classification
income_threshold_low = df['Total Income'].quantile(0.33)
income_threshold_high = df['Total Income'].quantile(0.67)

df['income_level'] = pd.cut(df['Total Income'], 
                            bins=[-np.inf, income_threshold_low, income_threshold_high, np.inf],
                            labels=['Low_Income', 'Middle_Income', 'High_Income'])
```

#### Step 2.2: Feature Selection

**Selected Features:**
- **Expenditure Categories:** Total food, housing, utilities, healthcare, education, transportation
- **Demographics:** Family size, number of workers, region (urban/rural)
- **Income Sources:** Main income, secondary income, remittances
- **Ratios:** Food-to-income ratio, savings rate (if available)

**Excluded:** Target variable itself, identification columns

```python
# Define feature columns
expenditure_cols = ['Total Food', 'Housing', 'Utilities', 'Healthcare', 
                    'Education', 'Transportation', 'Clothing', 'Personal Care']
demographic_cols = ['Family Size', 'Number of Workers', 'Region']
income_cols = ['Wage Income', 'Self-Employment Income', 'Other Income']

feature_cols = expenditure_cols + demographic_cols + income_cols
X = df[feature_cols]
y = df['income_level']  # or 'income_quintile'
```

#### Step 2.3: Data Preprocessing

**Normalization/Standardization:**
```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
```

**Why standardize?**
- Different features have different scales (income in thousands, family size in single digits)
- Required for SVM, distance-based methods (Week 3-4)
- Ensures fair feature contribution

**Handle Categorical Variables:**
```python
# Encode categorical features
le = LabelEncoder()
X_scaled['Region'] = le.fit_transform(X_scaled['Region'])
```

#### Step 2.4: Train-Test Split

```python
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                      test_size=0.2, 
                                                      random_state=42,
                                                      stratify=y)  # Maintain class distribution
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"\nClass distribution in training set:\n{y_train.value_counts(normalize=True)}")
```

#### Step 2.5: Classification Models

**Model 1: Logistic Regression (Week 3)**
```python
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
```

**Why:** Baseline model, interpretable, fast training, probability outputs

---

**Model 2: Support Vector Machine (Week 4)**
```python
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
```

**Why:** Handles high-dimensional data, good for multi-class problems, effective in practice

**Hyperparameters to tune:**
- `kernel`: 'linear', 'rbf', 'poly' (Week 5)
- `C`: Regularization strength (larger C = less regularization)
- `gamma`: Kernel coefficient (only for rbf/poly)

---

**Model 3: Gradient Boosting (Week 8 - Ensemble Learning)**
```python
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(n_estimators=100, 
                                      learning_rate=0.1,
                                      max_depth=5,
                                      random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
```

**Why:** Powerful ensemble method, handles feature interactions, provides feature importance

**Hyperparameters:**
- `n_estimators`: Number of boosting stages (100-300)
- `learning_rate`: Shrinkage/step size (0.01-0.1)
- `max_depth`: Tree depth (3-8)

---

**Model 4: Random Forest (Week 8 - Ensemble Learning)**
```python
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100,
                                  max_depth=10,
                                  min_samples_split=5,
                                  random_state=42,
                                  n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
```

**Why:** Robust, handles non-linear relationships, feature importance, parallel processing

#### Step 2.6: Model Evaluation Metrics

**Metrics to Report:**

1. **Accuracy:** Overall correctness
   $$\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$$

2. **Precision:** Of predicted positives, how many are correct
   $$\text{Precision} = \frac{TP}{TP + FP}$$

3. **Recall (Sensitivity):** Of actual positives, how many did we find
   $$\text{Recall} = \frac{TP}{TP + FN}$$

4. **F1-Score:** Harmonic mean of precision and recall
   $$F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$

5. **Confusion Matrix:** Shows TP, FP, TN, FN for each class

**Evaluation Code:**
```python
from sklearn.metrics import classification_report, confusion_matrix

# For each model
print(f"=== Model Performance ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
```

#### Step 2.7: Model Comparison

Create a comparison table of all models:
```python
results_dict = {
    'Logistic Regression': {
        'Accuracy': accuracy_score(y_test, y_pred_lr),
        'Precision': precision_score(y_test, y_pred_lr, average='weighted'),
        'Recall': recall_score(y_test, y_pred_lr, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred_lr, average='weighted')
    },
    'SVM': {
        'Accuracy': accuracy_score(y_test, y_pred_svm),
        'Precision': precision_score(y_test, y_pred_svm, average='weighted'),
        'Recall': recall_score(y_test, y_pred_svm, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred_svm, average='weighted')
    },
    'Gradient Boosting': {
        'Accuracy': accuracy_score(y_test, y_pred_gb),
        'Precision': precision_score(y_test, y_pred_gb, average='weighted'),
        'Recall': recall_score(y_test, y_pred_gb, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred_gb, average='weighted')
    },
    'Random Forest': {
        'Accuracy': accuracy_score(y_test, y_pred_rf),
        'Precision': precision_score(y_test, y_pred_rf, average='weighted'),
        'Recall': recall_score(y_test, y_pred_rf, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred_rf, average='weighted')
    }
}

results_df = pd.DataFrame(results_dict).T
print(results_df)
```

#### Step 2.8: Feature Importance (for tree-based models)

```python
# For Gradient Boosting and Random Forest
feature_importance_gb = pd.DataFrame({
    'Feature': X.columns,
    'Importance': gb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 10 Most Important Features:")
print(feature_importance_gb.head(10))

# Visualize
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_gb.head(10), x='Importance', y='Feature')
plt.title('Top 10 Feature Importance - Gradient Boosting')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()
```

---

## Section 3: Household Clustering and Segmentation (UNSUPERVISED LEARNING)

### Methodology Overview

**Goal:** Discover natural groupings of households with similar expenditure patterns without using income labels.

**Approach:**
1. Select relevant expenditure features
2. Standardize all features (critical for clustering)
3. Determine optimal number of clusters
4. Apply multiple clustering algorithms
5. Profile each cluster
6. Interpret household segments

### Detailed Steps

#### Step 3.1: Feature Selection for Clustering

**Features:** Expenditure categories normalized by total income

```python
# Prepare clustering data
expenditure_features = ['Total Food', 'Housing', 'Utilities', 'Healthcare', 
                       'Education', 'Transportation', 'Clothing', 'Personal Care']

# Create expenditure ratios (as percentage of total income)
X_cluster = df[expenditure_features].copy()
for col in expenditure_features:
    X_cluster[col] = X_cluster[col] / df['Total Income']

# Add demographic features
X_cluster['Family Size'] = df['Family Size']
X_cluster['Number of Workers'] = df['Number of Workers']

# Standardize features
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)
X_cluster_scaled = pd.DataFrame(X_cluster_scaled, columns=X_cluster.columns)
```

**Why these features?**
- Expenditure ratios are comparable across income levels
- Family size influences spending patterns
- Number of workers indicates earning capacity

#### Step 3.2: Determine Optimal Number of Clusters

**Method 1: Elbow Method**
```python
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Test different numbers of clusters
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(X_cluster_scaled)
    inertias.append(kmeans_temp.inertia_)
    silhouette_scores.append(silhouette_score(X_cluster_scaled, kmeans_temp.labels_))

# Plot elbow curve
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(K_range, inertias, 'bo-')
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia (Within-cluster SS)')
axes[0].set_title('Elbow Method')
axes[0].grid(True)

axes[1].plot(K_range, silhouette_scores, 'ro-')
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Analysis')
axes[1].grid(True)
plt.tight_layout()
plt.show()

print("Silhouette Scores:")
for k, score in zip(K_range, silhouette_scores):
    print(f"k={k}: {score:.4f}")
```

**Interpretation:**
- **Elbow point:** Where inertia decreases slower (typical k=3-5)
- **Silhouette Score:** Higher is better (range -1 to 1). Choose k with highest score.

**Method 2: Silhouette Coefficient**
- Measures how similar an object is to its own cluster vs other clusters
- Range: -1 (bad) to 1 (good)
- Values > 0.5 indicate well-separated clusters

#### Step 3.3: K-Means Clustering (Week 11)

**Algorithm Overview:**
1. Randomly initialize k centroids
2. Assign each point to nearest centroid
3. Update centroids as mean of assigned points
4. Repeat until convergence

```python
# Apply K-Means with optimal k (e.g., k=4)
optimal_k = 4  # Based on elbow/silhouette analysis
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10, max_iter=300)
df['Cluster_KMeans'] = kmeans.fit_predict(X_cluster_scaled)

print(f"K-Means Clustering (k={optimal_k})")
print(f"Cluster distribution:\n{df['Cluster_KMeans'].value_counts().sort_index()}")
print(f"Within-cluster sum of squares: {kmeans.inertia_:.2f}")
```

**Hyperparameters:**
- `n_clusters`: Number of clusters (from elbow method)
- `random_state`: For reproducibility
- `n_init`: Number of times to run (default=10)
- `max_iter`: Maximum iterations (default=300)

#### Step 3.4: Hierarchical Clustering (Week 11)

**Algorithm:** Agglomerative (bottom-up) approach

```python
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

# Compute linkage matrix
linkage_matrix = sch.linkage(X_cluster_scaled, method='ward')

# Plot dendrogram
plt.figure(figsize=(14, 7))
dendrogram = sch.dendrogram(linkage_matrix, 
                             leaf_font_size=10,
                             no_labels=False)
plt.title('Hierarchical Clustering Dendrogram (Ward Linkage)')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.axhline(y=50, c='r', linestyle='--', label='Cut-off line')  # Adjust height for k clusters
plt.legend()
plt.tight_layout()
plt.show()

# Apply Hierarchical Clustering
hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage='ward')
df['Cluster_Hierarchical'] = hierarchical.fit_predict(X_cluster_scaled)

print(f"Hierarchical Clustering (k={optimal_k})")
print(f"Cluster distribution:\n{df['Cluster_Hierarchical'].value_counts().sort_index()}")
```

**Linkage Methods:**
- `ward`: Minimizes variance (recommended)
- `complete`: Maximum distance between clusters
- `average`: Average distance between clusters
- `single`: Minimum distance (can create chain effect)

#### Step 3.5: DBSCAN Clustering (Week 11 - Density-based)

```python
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Find optimal eps using k-distance graph
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(X_cluster_scaled)
distances, indices = neighbors_fit.kneighbors(X_cluster_scaled)
distances = np.sort(distances[:, -1], axis=0)

plt.figure(figsize=(10, 5))
plt.plot(distances)
plt.ylabel('5-NN Distance')
plt.xlabel('Data Points sorted by distance')
plt.title('K-distance Graph for eps Selection')
plt.axhline(y=2.5, c='r', linestyle='--', label='Possible eps')
plt.legend()
plt.tight_layout()
plt.show()

# Apply DBSCAN with eps from k-distance graph
dbscan = DBSCAN(eps=2.5, min_samples=5)
df['Cluster_DBSCAN'] = dbscan.fit_predict(X_cluster_scaled)

n_clusters_db = len(set(df['Cluster_DBSCAN'])) - (1 if -1 in df['Cluster_DBSCAN'] else 0)
n_noise = list(df['Cluster_DBSCAN']).count(-1)

print(f"DBSCAN Clustering Results:")
print(f"Number of clusters: {n_clusters_db}")
print(f"Number of noise points: {n_noise}")
print(f"Cluster distribution:\n{df['Cluster_DBSCAN'].value_counts().sort_index()}")
```

**DBSCAN Parameters:**
- `eps`: Maximum distance between points (from k-distance graph)
- `min_samples`: Minimum points in neighborhood to form a cluster
- Returns -1 for noise points

#### Step 3.6: Cluster Profiling and Interpretation

```python
# Profile K-Means clusters
cluster_profiles = df.groupby('Cluster_KMeans')[expenditure_features].mean()
print("\n=== Cluster Profiles (Average Expenditure Ratios) ===")
print(cluster_profiles)

# Create interpretation
cluster_names = {
    0: 'Budget-Conscious Families',
    1: 'Balanced Spenders',
    2: 'Health & Education Focused',
    3: 'High-Income Spenders'
    # Adjust based on actual profiles
}

df['Cluster_Name'] = df['Cluster_KMeans'].map(cluster_names)

# Visualize cluster characteristics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for i, col in enumerate(expenditure_features[:4]):
    ax = axes[i//2, i%2]
    df.boxplot(column=col, by='Cluster_Name', ax=ax)
    ax.set_title(f'{col} by Cluster')
    ax.set_xlabel('Cluster')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
plt.suptitle('')
plt.tight_layout()
plt.show()

# Summary statistics by cluster
for cluster in sorted(df['Cluster_KMeans'].unique()):
    cluster_data = df[df['Cluster_KMeans'] == cluster]
    print(f"\n{'='*60}")
    print(f"Cluster {cluster}: {cluster_names[cluster]}")
    print(f"Size: {len(cluster_data)} households")
    print(f"Average Total Income: ₱{cluster_data['Total Income'].mean():.2f}")
    print(f"Average Family Size: {cluster_data['Family Size'].mean():.2f}")
    print(f"Average Workers per Household: {cluster_data['Number of Workers'].mean():.2f}")
    print(f"Top Spending Category: {cluster_data[expenditure_features].mean().idxmax()}")
```

---

## Section 4: Anomaly Detection (UNSUPERVISED LEARNING)

### Methodology Overview

**Goal:** Identify unusual household spending patterns that deviate significantly from normal behavior.

**Why important:**
- Detect data entry errors
- Identify extremely high/low spenders
- Find unusual budget allocations
- Potential fraud or special circumstances

**Approach:** Apply multiple anomaly detection methods and compare results

### Detailed Steps

#### Step 4.1: Isolation Forest (Week 11 - Density-based Anomaly Detection)

**Algorithm Concept:**
- Isolates anomalies by randomly selecting features and thresholds
- Anomalies are isolated faster than normal points
- Anomalies have shorter path lengths

```python
from sklearn.ensemble import IsolationForest

# Apply Isolation Forest
iso_forest = IsolationForest(contamination=0.05,  # Expect 5% anomalies
                             random_state=42,
                             n_estimators=100)
anomaly_if = iso_forest.fit_predict(X_cluster_scaled)
anomaly_score_if = iso_forest.score_samples(X_cluster_scaled)

df['Anomaly_IsolationForest'] = anomaly_if
df['Anomaly_Score_IF'] = anomaly_score_if

n_anomalies_if = sum(anomaly_if == -1)
print(f"Isolation Forest:")
print(f"Number of anomalies detected: {n_anomalies_if}")
print(f"Percentage: {(n_anomalies_if/len(df))*100:.2f}%")
```

**Parameters:**
- `contamination`: Expected proportion of anomalies (default=0.1)
- `random_state`: For reproducibility
- `n_estimators`: Number of isolation trees (default=100)

**Output:** -1 for anomalies, 1 for normal points

#### Step 4.2: Local Outlier Factor (LOF) (Week 11 - Density-based)

**Algorithm Concept:**
- Compares local density of a point to its neighbors
- Points with much lower density than neighbors are outliers
- Good for varying density regions

```python
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
anomaly_lof = lof.fit_predict(X_cluster_scaled)
anomaly_score_lof = lof.negative_outlier_factor_

df['Anomaly_LOF'] = anomaly_lof
df['Anomaly_Score_LOF'] = anomaly_score_lof

n_anomalies_lof = sum(anomaly_lof == -1)
print(f"\nLocal Outlier Factor (LOF):")
print(f"Number of anomalies detected: {n_anomalies_lof}")
print(f"Percentage: {(n_anomalies_lof/len(df))*100:.2f}%")
```

**Parameters:**
- `n_neighbors`: Number of neighbors to consider (default=20)
- `contamination`: Expected proportion of anomalies

#### Step 4.3: Mahalanobis Distance / Statistical Method

```python
from scipy.spatial.distance import mahalanobis
from scipy import stats

# Calculate mean and covariance
mean = np.mean(X_cluster_scaled, axis=0)
cov = np.cov(X_cluster_scaled.T)

# Calculate Mahalanobis distance for each point
mahal_dist = []
for i in range(len(X_cluster_scaled)):
    diff = X_cluster_scaled[i] - mean
    mahal_dist.append(np.sqrt(diff.T @ np.linalg.inv(cov) @ diff))

df['Mahal_Distance'] = mahal_dist

# Threshold: Chi-square distribution with df = number of features
threshold = stats.chi2.ppf(0.95, df=X_cluster_scaled.shape[1])
anomaly_mahal = [1 if d > threshold else -1 for d in mahal_dist]
df['Anomaly_Mahal'] = anomaly_mahal

n_anomalies_mahal = sum(np.array(anomaly_mahal) == -1)
print(f"\nMahalanobis Distance:")
print(f"Number of anomalies detected: {n_anomalies_mahal}")
print(f"Percentage: {(n_anomalies_mahal/len(df))*100:.2f}%")
```

#### Step 4.4: Z-Score Based Anomaly Detection

```python
# Calculate Z-scores
z_scores = np.abs(stats.zscore(X_cluster_scaled))

# Flag as anomaly if any feature has |z-score| > 3
anomaly_zscore = [-1 if (z_scores[i] > 3).any() else 1 for i in range(len(z_scores))]
df['Anomaly_ZScore'] = anomaly_zscore

n_anomalies_zscore = sum(np.array(anomaly_zscore) == -1)
print(f"\nZ-Score Method:")
print(f"Number of anomalies detected: {n_anomalies_zscore}")
print(f"Percentage: {(n_anomalies_zscore/len(df))*100:.2f}%")
```

**Threshold Interpretation:**
- $|z| > 2$: Moderately unusual (95% confidence)
- $|z| > 3$: Very unusual (99.7% confidence)

#### Step 4.5: Consensus Anomaly Detection

```python
# Count how many methods flagged each point as anomaly
anomaly_count = (
    (df['Anomaly_IsolationForest'] == -1).astype(int) +
    (df['Anomaly_LOF'] == -1).astype(int) +
    (df['Anomaly_Mahal'] == -1).astype(int) +
    (df['Anomaly_ZScore'] == -1).astype(int)
)

df['Anomaly_Count'] = anomaly_count
df['Is_Anomaly_Consensus'] = (anomaly_count >= 2).astype(int)  # Flagged by 2+ methods

consensus_anomalies = df[df['Is_Anomaly_Consensus'] == 1]
print(f"\nConsensus Anomaly Detection (2+ methods agree):")
print(f"Number of anomalies: {len(consensus_anomalies)}")
print(f"Percentage: {(len(consensus_anomalies)/len(df))*100:.2f}%")

# Method agreement
print("\nMethod Agreement Summary:")
print(f"Flagged by 1 method: {sum(anomaly_count == 1)}")
print(f"Flagged by 2 methods: {sum(anomaly_count == 2)}")
print(f"Flagged by 3 methods: {sum(anomaly_count == 3)}")
print(f"Flagged by all 4 methods: {sum(anomaly_count == 4)}")
```

#### Step 4.6: Anomaly Visualization

```python
# Visualize anomalies using scatter plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Isolation Forest
scatter1 = axes[0, 0].scatter(X_cluster_scaled.iloc[:, 0], 
                              X_cluster_scaled.iloc[:, 1],
                              c=df['Anomaly_IsolationForest'],
                              cmap='RdYlGn', s=50, alpha=0.6)
axes[0, 0].set_title('Isolation Forest Anomalies')
axes[0, 0].set_xlabel('Feature 1')
axes[0, 0].set_ylabel('Feature 2')
plt.colorbar(scatter1, ax=axes[0, 0])

# Plot 2: LOF
scatter2 = axes[0, 1].scatter(X_cluster_scaled.iloc[:, 0],
                              X_cluster_scaled.iloc[:, 1],
                              c=df['Anomaly_LOF'],
                              cmap='RdYlGn', s=50, alpha=0.6)
axes[0, 1].set_title('Local Outlier Factor Anomalies')
axes[0, 1].set_xlabel('Feature 1')
axes[0, 1].set_ylabel('Feature 2')
plt.colorbar(scatter2, ax=axes[0, 1])

# Plot 3: Mahalanobis Distance
scatter3 = axes[1, 0].scatter(X_cluster_scaled.iloc[:, 0],
                              X_cluster_scaled.iloc[:, 1],
                              c=df['Mahal_Distance'],
                              cmap='viridis', s=50, alpha=0.6)
axes[1, 0].set_title('Mahalanobis Distance')
axes[1, 0].set_xlabel('Feature 1')
axes[1, 0].set_ylabel('Feature 2')
plt.colorbar(scatter3, ax=axes[1, 0])

# Plot 4: Consensus
scatter4 = axes[1, 1].scatter(X_cluster_scaled.iloc[:, 0],
                              X_cluster_scaled.iloc[:, 1],
                              c=df['Anomaly_Count'],
                              cmap='YlOrRd', s=50, alpha=0.6)
axes[1, 1].set_title('Consensus Anomaly Score (# Methods)')
axes[1, 1].set_xlabel('Feature 1')
axes[1, 1].set_ylabel('Feature 2')
plt.colorbar(scatter4, ax=axes[1, 1])

plt.tight_layout()
plt.show()
```

#### Step 4.7: Analyze Anomalous Households

```python
# Examine consensus anomalies
print("\n=== TOP 10 ANOMALOUS HOUSEHOLDS ===")
top_anomalies = df.nlargest(10, 'Anomaly_Count')[
    ['Total Income', 'Total Food', 'Housing', 'Healthcare', 'Education',
     'Anomaly_Count', 'Anomaly_Score_IF', 'Mahal_Distance']
]
print(top_anomalies)

# Compare anomalies vs normal households
print("\n=== ANOMALIES vs NORMAL HOUSEHOLDS ===")
anomaly_comparison = pd.DataFrame({
    'Normal_Mean': df[df['Is_Anomaly_Consensus'] == 0][expenditure_features].mean(),
    'Anomaly_Mean': df[df['Is_Anomaly_Consensus'] == 1][expenditure_features].mean(),
})
anomaly_comparison['Difference'] = anomaly_comparison['Anomaly_Mean'] - anomaly_comparison['Normal_Mean']
print(anomaly_comparison)

# Characteristics of anomalies
anomaly_data = df[df['Is_Anomaly_Consensus'] == 1]
normal_data = df[df['Is_Anomaly_Consensus'] == 0]

print("\nCharacteristics Comparison:")
print(f"Anomalies - Avg Income: ₱{anomaly_data['Total Income'].mean():.2f}")
print(f"Normal    - Avg Income: ₱{normal_data['Total Income'].mean():.2f}")
print(f"\nAnomalies - Avg Family Size: {anomaly_data['Family Size'].mean():.2f}")
print(f"Normal    - Avg Family Size: {normal_data['Family Size'].mean():.2f}")

---

## Section 5: Dimensionality Reduction and Visualization

### Methodology Overview

**Goal:** Reduce 60 dimensions to 2-3 dimensions for visualization while preserving structure and information.

**Why important:**
- Visualize high-dimensional data
- Understand cluster separation
- Identify natural groupings
- Reduce noise and computational cost

### Detailed Steps

#### Step 5.1: Principal Component Analysis (PCA) - Week 9

**Algorithm Concept:**
- Linear transformation that finds principal components (directions of max variance)
- First PC captures most variance, second PC captures second-most, etc.
- Uncorrelated components

**Mathematical Foundation:**
$$X_{\text{transformed}} = X \cdot W$$

where $W$ contains eigenvectors of the covariance matrix sorted by eigenvalues in descending order.

```python
from sklearn.decomposition import PCA

# Apply PCA with 2 components for visualization
pca = PCA(n_components=2, random_state=42)
X_pca_2d = pca.fit_transform(X_cluster_scaled)

# Store results
df['PCA_1'] = X_pca_2d[:, 0]
df['PCA_2'] = X_pca_2d[:, 1]

# Explained variance
print("=== PCA Results ===")
print(f"Explained Variance Ratio:")
print(f"PC1: {pca.explained_variance_ratio_[0]:.4f}")
print(f"PC2: {pca.explained_variance_ratio_[1]:.4f}")
print(f"Total (2 components): {pca.explained_variance_ratio_.sum():.4f}")

# Check how many components needed for 95% variance
pca_full = PCA(random_state=42)
pca_full.fit(X_cluster_scaled)
cumsum_var = np.cumsum(pca_full.explained_variance_ratio_)
n_components_95 = np.argmax(cumsum_var >= 0.95) + 1
print(f"\nNumber of components needed for 95% variance: {n_components_95}")

# Visualize explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(1, len(pca_full.explained_variance_ratio_) + 1),
            pca_full.explained_variance_ratio_)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('PCA Explained Variance per Component')
axes[0].set_xticks(range(1, min(11, len(pca_full.explained_variance_ratio_) + 1)))

axes[1].plot(range(1, len(cumsum_var) + 1), cumsum_var, 'bo-')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Explained Variance')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# PCA loadings (feature contributions to each PC)
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=X_cluster_scaled.columns
)
print("\nTop 5 Features Contributing to Each Component:")
print("PC1 - Top Features:")
print(loadings['PC1'].nlargest(5))
print("\nPC2 - Top Features:")
print(loadings['PC2'].nlargest(5))
```

**PCA Hyperparameters:**
- `n_components`: Number of dimensions to reduce to
- Can be: integer (number of components), float (variance explained), or 'mle'

#### Step 5.2: Visualize Clusters in PCA Space

```python
# Plot clusters using PCA
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: K-Means Clusters
scatter1 = axes[0, 0].scatter(df['PCA_1'], df['PCA_2'],
                              c=df['Cluster_KMeans'],
                              cmap='viridis', s=50, alpha=0.6)
axes[0, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[0, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
axes[0, 0].set_title('K-Means Clusters in PCA Space')
axes[0, 0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0, 0], label='Cluster')

# Plot 2: Hierarchical Clusters
scatter2 = axes[0, 1].scatter(df['PCA_1'], df['PCA_2'],
                              c=df['Cluster_Hierarchical'],
                              cmap='viridis', s=50, alpha=0.6)
axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
axes[0, 1].set_title('Hierarchical Clusters in PCA Space')
axes[0, 1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[0, 1], label='Cluster')

# Plot 3: Income Level
scatter3 = axes[1, 0].scatter(df['PCA_1'], df['PCA_2'],
                              c=df['Total Income'],
                              cmap='RdYlGn', s=50, alpha=0.6)
axes[1, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[1, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
axes[1, 0].set_title('Household Income in PCA Space')
axes[1, 0].grid(True, alpha=0.3)
plt.colorbar(scatter3, ax=axes[1, 0], label='Total Income (₱)')

# Plot 4: Anomalies
scatter4 = axes[1, 1].scatter(df['PCA_1'], df['PCA_2'],
                              c=df['Anomaly_Count'],
                              cmap='YlOrRd', s=50, alpha=0.6)
axes[1, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[1, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
axes[1, 1].set_title('Anomaly Consensus Score in PCA Space')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(scatter4, ax=axes[1, 1], label='# Methods')

plt.tight_layout()
plt.show()
```

#### Step 5.3: t-SNE (t-Distributed Stochastic Neighbor Embedding) - Week 10

**Algorithm Concept:**
- Non-linear dimensionality reduction
- Preserves local neighborhood structure better than PCA
- Good for visualization of complex, multi-scale structures
- Computationally more intensive than PCA

```python
from sklearn.manifold import TSNE

# Apply t-SNE (warning: may take time on large datasets)
print("Applying t-SNE (this may take a few minutes)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000, verbose=1)
X_tsne = tsne.fit_transform(X_cluster_scaled)

df['TSNE_1'] = X_tsne[:, 0]
df['TSNE_2'] = X_tsne[:, 1]

print("t-SNE completed!")
```

**t-SNE Hyperparameters:**
- `n_components`: Usually 2 or 3 for visualization
- `perplexity`: Expected number of neighbors (5-50, default=30)
- `n_iter`: Number of iterations (default=1000)
- `learning_rate`: Learning rate (default=200)

#### Step 5.4: Visualize with t-SNE

```python
# Plot t-SNE visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: K-Means Clusters
scatter1 = axes[0, 0].scatter(df['TSNE_1'], df['TSNE_2'],
                              c=df['Cluster_KMeans'],
                              cmap='viridis', s=50, alpha=0.6)
axes[0, 0].set_xlabel('t-SNE 1')
axes[0, 0].set_ylabel('t-SNE 2')
axes[0, 0].set_title('K-Means Clusters in t-SNE Space')
axes[0, 0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0, 0], label='Cluster')

# Plot 2: Income Level
scatter2 = axes[0, 1].scatter(df['TSNE_1'], df['TSNE_2'],
                              c=df['Total Income'],
                              cmap='RdYlGn', s=50, alpha=0.6)
axes[0, 1].set_xlabel('t-SNE 1')
axes[0, 1].set_ylabel('t-SNE 2')
axes[0, 1].set_title('Household Income in t-SNE Space')
axes[0, 1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[0, 1], label='Total Income (₱)')

# Plot 3: Anomalies
scatter3 = axes[1, 0].scatter(df['TSNE_1'], df['TSNE_2'],
                              c=df['Anomaly_Count'],
                              cmap='YlOrRd', s=50, alpha=0.6)
axes[1, 0].set_xlabel('t-SNE 1')
axes[1, 0].set_ylabel('t-SNE 2')
axes[1, 0].set_title('Anomaly Detection in t-SNE Space')
axes[1, 0].grid(True, alpha=0.3)
plt.colorbar(scatter3, ax=axes[1, 0], label='# Methods')

# Plot 4: Family Size
scatter4 = axes[1, 1].scatter(df['TSNE_1'], df['TSNE_2'],
                              c=df['Family Size'],
                              cmap='cool', s=50, alpha=0.6)
axes[1, 1].set_xlabel('t-SNE 1')
axes[1, 1].set_ylabel('t-SNE 2')
axes[1, 1].set_title('Family Size in t-SNE Space')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(scatter4, ax=axes[1, 1], label='Family Size')

plt.tight_layout()
plt.show()
```

#### Step 5.5: 3D Visualization (Optional)

```python
# 3D PCA
pca_3d = PCA(n_components=3, random_state=42)
X_pca_3d = pca_3d.fit_transform(X_cluster_scaled)

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(X_pca_3d[:, 0], X_pca_3d[:, 1], X_pca_3d[:, 2],
                     c=df['Cluster_KMeans'],
                     cmap='viridis', s=30, alpha=0.6)

ax.set_xlabel(f'PC1 ({pca_3d.explained_variance_ratio_[0]:.1%})')
ax.set_ylabel(f'PC2 ({pca_3d.explained_variance_ratio_[1]:.1%})')
ax.set_zlabel(f'PC3 ({pca_3d.explained_variance_ratio_[2]:.1%})')
ax.set_title('3D PCA Visualization of Clusters')

plt.colorbar(scatter, ax=ax, label='Cluster', pad=0.1)
plt.tight_layout()
plt.show()

print(f"Explained Variance (3D): {pca_3d.explained_variance_ratio_.sum():.4f}")
```

#### Step 5.6: Comparison of Dimensionality Reduction Methods

```python
# Create comparison plot
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# PCA vs t-SNE
axes[0].scatter(df['PCA_1'], df['PCA_2'], c=df['Cluster_KMeans'], 
               cmap='viridis', s=50, alpha=0.6)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
axes[0].set_title('PCA Visualization')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(df['TSNE_1'], df['TSNE_2'], c=df['Cluster_KMeans'],
               cmap='viridis', s=50, alpha=0.6)
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].set_title('t-SNE Visualization')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Comparison Notes:")
print("PCA:")
print(f"  - Explains {pca.explained_variance_ratio_.sum():.1%} of variance with 2 components")
print(f"  - Linear transformation")
print(f"  - Fast to compute")
print("\nt-SNE:")
print("  - Non-linear transformation")
print("  - Better local neighborhood preservation")
print("  - Slower to compute")
print("  - Better for visualization of complex structures")

---

## Section 6: Model Evaluation and Results Summary

### Methodology Overview

**Goal:** Comprehensive comparison and summary of all analysis components with actionable insights.

### Step 6.1: Classification Model Comparison

```python
# Create comprehensive model comparison
print("="*80)
print("CLASSIFICATION MODEL PERFORMANCE SUMMARY")
print("="*80)

models_summary = pd.DataFrame({
    'Logistic Regression': {
        'Accuracy': accuracy_score(y_test, y_pred_lr),
        'Precision (weighted)': precision_score(y_test, y_pred_lr, average='weighted'),
        'Recall (weighted)': recall_score(y_test, y_pred_lr, average='weighted'),
        'F1-Score (weighted)': f1_score(y_test, y_pred_lr, average='weighted'),
        'Training Time': 'Fast',
        'Interpretability': 'High'
    },
    'SVM': {
        'Accuracy': accuracy_score(y_test, y_pred_svm),
        'Precision (weighted)': precision_score(y_test, y_pred_svm, average='weighted'),
        'Recall (weighted)': recall_score(y_test, y_pred_svm, average='weighted'),
        'F1-Score (weighted)': f1_score(y_test, y_pred_svm, average='weighted'),
        'Training Time': 'Medium',
        'Interpretability': 'Low'
    },
    'Gradient Boosting': {
        'Accuracy': accuracy_score(y_test, y_pred_gb),
        'Precision (weighted)': precision_score(y_test, y_pred_gb, average='weighted'),
        'Recall (weighted)': recall_score(y_test, y_pred_gb, average='weighted'),
        'F1-Score (weighted)': f1_score(y_test, y_pred_gb, average='weighted'),
        'Training Time': 'Medium-Slow',
        'Interpretability': 'Medium'
    },
    'Random Forest': {
        'Accuracy': accuracy_score(y_test, y_pred_rf),
        'Precision (weighted)': precision_score(y_test, y_pred_rf, average='weighted'),
        'Recall (weighted)': recall_score(y_test, y_pred_rf, average='weighted'),
        'F1-Score (weighted)': f1_score(y_test, y_pred_rf, average='weighted'),
        'Training Time': 'Medium',
        'Interpretability': 'Medium'
    }
})

print(models_summary.T)

# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

metrics_df = models_summary.T[['Accuracy', 'Precision (weighted)', 'Recall (weighted)', 'F1-Score (weighted)']]
metrics_df.plot(kind='bar', ax=axes[0])
axes[0].set_title('Classification Model Performance Metrics')
axes[0].set_ylabel('Score')
axes[0].set_xlabel('Model')
axes[0].legend(loc='lower right', fontsize=8)
axes[0].set_ylim([0, 1.05])
axes[0].grid(True, alpha=0.3)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45)

# Accuracy comparison
models_list = ['Logistic\nRegression', 'SVM', 'Gradient\nBoosting', 'Random\nForest']
accuracies = [
    accuracy_score(y_test, y_pred_lr),
    accuracy_score(y_test, y_pred_svm),
    accuracy_score(y_test, y_pred_gb),
    accuracy_score(y_test, y_pred_rf)
]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
bars = axes[1].bar(models_list, accuracies, color=colors, alpha=0.7)
axes[1].set_title('Model Accuracy Comparison')
axes[1].set_ylabel('Accuracy')
axes[1].set_ylim([0, 1.05])
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()
```

### Step 6.2: Clustering Quality Metrics

```python
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

print("\n" + "="*80)
print("CLUSTERING QUALITY METRICS")
print("="*80)

# Calculate metrics for K-Means
silhouette_km = silhouette_score(X_cluster_scaled, df['Cluster_KMeans'])
davies_bouldin_km = davies_bouldin_score(X_cluster_scaled, df['Cluster_KMeans'])
calinski_harabasz_km = calinski_harabasz_score(X_cluster_scaled, df['Cluster_KMeans'])

print(f"\nK-Means Clustering (k={optimal_k}):")
print(f"  Silhouette Score: {silhouette_km:.4f} (range: -1 to 1, higher is better)")
print(f"  Davies-Bouldin Index: {davies_bouldin_km:.4f} (lower is better)")
print(f"  Calinski-Harabasz Index: {calinski_harabasz_km:.4f} (higher is better)")

# For Hierarchical
silhouette_hier = silhouette_score(X_cluster_scaled, df['Cluster_Hierarchical'])
davies_bouldin_hier = davies_bouldin_score(X_cluster_scaled, df['Cluster_Hierarchical'])

print(f"\nHierarchical Clustering (k={optimal_k}):")
print(f"  Silhouette Score: {silhouette_hier:.4f}")
print(f"  Davies-Bouldin Index: {davies_bouldin_hier:.4f}")

# Comparison
clustering_comparison = pd.DataFrame({
    'K-Means': {
        'Silhouette': silhouette_km,
        'Davies-Bouldin': davies_bouldin_km,
        'Calinski-Harabasz': calinski_harabasz_km,
        'Interpretability': 'Easy',
        'Computational Cost': 'Low'
    },
    'Hierarchical': {
        'Silhouette': silhouette_hier,
        'Davies-Bouldin': davies_bouldin_hier,
        'Calinski-Harabasz': calinski_harabasz_score(X_cluster_scaled, df['Cluster_Hierarchical']),
        'Interpretability': 'Medium (Dendrogram)',
        'Computational Cost': 'Medium'
    }
})

print("\nClustering Algorithm Comparison:")
print(clustering_comparison.T)
```

### Step 6.3: Anomaly Detection Summary

```python
print("\n" + "="*80)
print("ANOMALY DETECTION SUMMARY")
print("="*80)

anomaly_summary = pd.DataFrame({
    'Method': ['Isolation Forest', 'Local Outlier Factor', 'Mahalanobis Distance', 'Z-Score'],
    'Anomalies Detected': [
        sum(df['Anomaly_IsolationForest'] == -1),
        sum(df['Anomaly_LOF'] == -1),
        sum(df['Anomaly_Mahal'] == -1),
        sum(df['Anomaly_ZScore'] == -1)
    ],
    'Percentage': [
        (sum(df['Anomaly_IsolationForest'] == -1) / len(df)) * 100,
        (sum(df['Anomaly_LOF'] == -1) / len(df)) * 100,
        (sum(df['Anomaly_Mahal'] == -1) / len(df)) * 100,
        (sum(df['Anomaly_ZScore'] == -1) / len(df)) * 100
    ]
})

print("\nAnomalies by Method:")
print(anomaly_summary.to_string(index=False))

print(f"\nConsensus Anomalies (2+ methods agree): {len(consensus_anomalies)} ({(len(consensus_anomalies)/len(df))*100:.2f}%)")

# Visualize anomaly detection results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Method comparison
methods = anomaly_summary['Method']
counts = anomaly_summary['Anomalies Detected']
colors_anom = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(methods)))
axes[0].bar(methods, counts, color=colors_anom, alpha=0.7)
axes[0].set_title('Anomalies Detected by Method')
axes[0].set_ylabel('Number of Anomalies')
axes[0].set_ylim([0, max(counts) * 1.1])
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')
axes[0].grid(True, alpha=0.3, axis='y')

# Consensus distribution
consensus_counts = df['Anomaly_Count'].value_counts().sort_index()
axes[1].bar(consensus_counts.index, consensus_counts.values, 
           color=plt.cm.YlOrRd(np.linspace(0.3, 0.9, len(consensus_counts))), alpha=0.7)
axes[1].set_xlabel('Number of Methods Flagging as Anomaly')
axes[1].set_ylabel('Number of Households')
axes[1].set_title('Anomaly Consensus Distribution')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()
```

### Step 6.4: Key Findings and Insights

```python
print("\n" + "="*80)
print("KEY FINDINGS AND INSIGHTS")
print("="*80)

print("\n1. INCOME CLASSIFICATION:")
print(f"   - Best performing model: [INSERT BEST MODEL NAME]")
print(f"   - Accuracy: [INSERT ACCURACY]%")
print(f"   - Key income drivers: [INSERT TOP 3 FEATURES]")
print(f"   - Model can reliably predict income level based on expenditure patterns")

print("\n2. HOUSEHOLD CLUSTERING:")
print(f"   - Optimal number of clusters: {optimal_k}")
print(f"   - Silhouette score: {silhouette_km:.4f}")
for i in range(optimal_k):
    cluster_size = len(df[df['Cluster_KMeans'] == i])
    avg_income = df[df['Cluster_KMeans'] == i]['Total Income'].mean()
    print(f"   - Cluster {i}: {cluster_size} households, avg income: ₱{avg_income:.2f}")

print("\n3. ANOMALY DETECTION:")
print(f"   - Consensus anomalies: {len(consensus_anomalies)} households ({(len(consensus_anomalies)/len(df))*100:.2f}%)")
print(f"   - Anomalies are {(anomaly_data['Total Income'].mean() / normal_data['Total Income'].mean()):.2f}x [higher/lower] income")
print(f"   - Potential data quality issues: {sum(df['Anomaly_Count'] == 4)} (all 4 methods agree)")

print("\n4. DIMENSIONALITY REDUCTION:")
print(f"   - PCA: {pca.explained_variance_ratio_.sum():.1%} variance with 2 components")
print(f"   - Top features: [PC1 and PC2 loadings]")
print(f"   - t-SNE shows clear cluster separation")

print("\n5. POLICY IMPLICATIONS:")
print("   - Identified household segments for targeted assistance")
print("   - Income predictability from spending patterns enables better profiling")
print("   - Anomaly detection can flag data quality issues or special cases")
```

### Step 6.5: Final Recommendations

```python
print("\n" + "="*80)
print("RECOMMENDATIONS FOR FUTURE WORK")
print("="*80)

recommendations = {
    'Data Collection': [
        'Collect more demographic variables',
        'Track temporal changes in household spending',
        'Include geographic location details'
    ],
    'Model Improvement': [
        'Perform hyperparameter tuning (Week 5)',
        'Use cross-validation for robust evaluation',
        'Try ensemble methods combining multiple algorithms'
    ],
    'Business Applications': [
        'Use income prediction for targeted programs',
        'Design household-specific financial assistance',
        'Monitor anomalies for data quality control'
    ],
    'Further Analysis': [
        'Analyze cluster transitions over time',
        'Investigate reasons for anomalies',
        'Compare predictions across regions'
    ]
}

for category, items in recommendations.items():
    print(f"\n{category}:")
    for i, item in enumerate(items, 1):
        print(f"  {i}. {item}")
```

### Summary Visualization Dashboard

```python
# Create a comprehensive summary dashboard
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Model Accuracy
ax1 = fig.add_subplot(gs[0, 0])
models = ['LR', 'SVM', 'GB', 'RF']
accs = [accuracy_score(y_test, y_pred_lr), 
        accuracy_score(y_test, y_pred_svm),
        accuracy_score(y_test, y_pred_gb),
        accuracy_score(y_test, y_pred_rf)]
ax1.bar(models, accs, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'], alpha=0.7)
ax1.set_title('Classification Accuracy')
ax1.set_ylim([0, 1])
ax1.grid(True, alpha=0.3, axis='y')

# 2. Cluster Distribution
ax2 = fig.add_subplot(gs[0, 1])
cluster_counts = df['Cluster_KMeans'].value_counts().sort_index()
ax2.bar(cluster_counts.index, cluster_counts.values, color=plt.cm.viridis(np.linspace(0, 1, optimal_k)), alpha=0.7)
ax2.set_title(f'Cluster Distribution (k={optimal_k})')
ax2.set_xlabel('Cluster')
ax2.set_ylabel('Count')
ax2.grid(True, alpha=0.3, axis='y')

# 3. Anomaly Methods
ax3 = fig.add_subplot(gs[0, 2])
methods = ['IF', 'LOF', 'Maha', 'Z-Score']
anom_counts = [sum(df['Anomaly_IsolationForest'] == -1),
               sum(df['Anomaly_LOF'] == -1),
               sum(df['Anomaly_Mahal'] == -1),
               sum(df['Anomaly_ZScore'] == -1)]
ax3.bar(methods, anom_counts, color=plt.cm.RdYlGn(np.linspace(0.2, 0.8, 4)), alpha=0.7)
ax3.set_title('Anomalies by Detection Method')
ax3.set_ylabel('Count')
ax3.grid(True, alpha=0.3, axis='y')

# 4. PCA Explained Variance
ax4 = fig.add_subplot(gs[1, 0])
ax4.bar(range(1, 6), pca_full.explained_variance_ratio_[:5])
ax4.set_title('PCA Explained Variance')
ax4.set_xlabel('Component')
ax4.set_ylabel('Variance Ratio')
ax4.grid(True, alpha=0.3, axis='y')

# 5. PCA Visualization
ax5 = fig.add_subplot(gs[1, 1])
scatter = ax5.scatter(df['PCA_1'], df['PCA_2'], c=df['Cluster_KMeans'], cmap='viridis', s=30, alpha=0.6)
ax5.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
ax5.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
ax5.set_title('Clusters in PCA Space')
ax5.grid(True, alpha=0.3)

# 6. Income Distribution
ax6 = fig.add_subplot(gs[1, 2])
ax6.hist(df['Total Income'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
ax6.set_title('Income Distribution')
ax6.set_xlabel('Total Income (₱)')
ax6.set_ylabel('Frequency')
ax6.grid(True, alpha=0.3, axis='y')

# 7. Feature Importance (Top 5)
ax7 = fig.add_subplot(gs[2, :2])
top_features = feature_importance_gb.head(5)
ax7.barh(top_features['Feature'], top_features['Importance'], color='teal', alpha=0.7)
ax7.set_title('Top 5 Feature Importance (Gradient Boosting)')
ax7.set_xlabel('Importance')
ax7.grid(True, alpha=0.3, axis='x')

# 8. Anomaly Consensus
ax8 = fig.add_subplot(gs[2, 2])
consensus_dist = df['Anomaly_Count'].value_counts().sort_index()
ax8.bar(consensus_dist.index, consensus_dist.values, color=plt.cm.YlOrRd(np.linspace(0.3, 0.9, len(consensus_dist))), alpha=0.7)
ax8.set_title('Anomaly Consensus')
ax8.set_xlabel('# Methods')
ax8.set_ylabel('Count')
ax8.grid(True, alpha=0.3, axis='y')

plt.suptitle('FIES Analysis - Comprehensive Summary Dashboard', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
```

---

## Summary Table: Methods Used by AI 221 Week

| Week | Topic | Methods Used |
|------|-------|--------------|
| 2 | EDA | Data exploration and visualization |
| 3 | Linear Regression | Logistic Regression (baseline classifier) |
| 4 | Kernel Methods | Support Vector Machine (SVM) |
| 7 | Neural Networks | Optional: MLP classifier |
| 8 | Ensemble Learning | Random Forest, Gradient Boosting |
| 9 | Linear Dim Reduction | PCA for visualization |
| 10 | Nonlinear Dim Reduction | t-SNE for visualization |
| 11 | Clustering & Anomaly | K-Means, Hierarchical, DBSCAN, Isolation Forest, LOF |

---

## Expected Outcomes

✅ **Supervised Learning:** Income level classification with >80% accuracy  
✅ **Unsupervised Learning:** 4-5 distinct household clusters identified  
✅ **Anomaly Detection:** 3-5% consensus anomalies detected  
✅ **Visualization:** Clear separation of clusters in PCA/t-SNE space  
✅ **Insights:** Actionable recommendations for policy makers